In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

In [2]:
import matplotlib.pyplot as plt
import jax.numpy as np
plt.rcParams.update({'font.size': 25})

In [3]:
%cd ../..

/n/home10/rdeshpande/morphogenesis/jax-morph


In [4]:
from jax import random, vmap
from jax_md import space, quantity, util
import jax_md.dataclasses as jdc

########## IMPORT JAX-MORPH FUNCTIONS ##########
################################################

from jax_morph.datastructures import SpaceFunc
from jax_morph.utils import _maybe_array, logistic
from jax_morph.simulation import simulation, sim_trajectory

from jax_morph.division_and_growth.cell_division import S_cell_division, S_cell_div_indep, S_cell_div_indep_MC
from jax_morph.division_and_growth.cell_growth import S_grow_cells

from jax_morph.mechanics.morse import S_mech_morse_relax
from jax_morph.cell_internals.stress import S_set_stress
from jax_morph.chemicals.secdiff import S_ss_chemfield

from jax_morph.cell_internals.divrates import S_set_divrate, div_nn
from jax_morph.cell_internals.secretion import sec_nn
from jax_morph.cell_internals.grad_estimate import S_chemical_gradients
from jax_morph.cell_internals.hidden_state import hidden_state_nn, S_hidden_state

from jax_morph.initial_states import init_state_grow
from jax_morph.visualization import draw_circles_ctype, draw_circles_chem, draw_circles_divrate, draw_circles
from Ramya.mech_homogeneous_growth.chemical import S_fixed_chemfield

from functools import partial
import equinox as eqx
import haiku as hk

# For saving data
import pickle
import os

# Utils for easier organization
from Ramya.utils.gene_utils import default_params, build_sim_from_params, run_optimization
from Ramya.utils.gene_utils import position_sum_of_squares, v_metric

In [5]:
import warnings
warnings.filterwarnings('ignore')
from Francesco.ALIFE_plots import alife_utils
from Ramya.utils import gene_utils
from copy import deepcopy
import networkx as nx

In [6]:
%load_ext autoreload
%autoreload 2

In [10]:
%ls ../data/paper

analysis_francesco.ipynb
analysis.ipynb
elongation_100_loss.pkl
elongation_100_params.pkl
elongation_1.pkl
elongation_2_loss.pkl
elongation_2_params.pkl
elongation_decay_0.0_2_loss.pkl
elongation_decay_0.0_2_params.pkl
elongation_decay_0.0_50_loss.pkl
elongation_decay_0.0_50_params.pkl
elongation_decay_0.8_30_loss.pkl
elongation_decay_0.8_30_params.pkl
figures/
homogeneous_growth_100_hidden_8_grads
homogeneous_growth_100_hidden_8_grads_power_law
homogeneous_growth_100_hidden_8_loss
homogeneous_growth_100_hidden_8_loss_power_law
homogeneous_growth_100_hidden_8_params
homogeneous_growth_100_hidden_8_params_power_law
vshape_100_loss.pkl
vshape_100_params.pkl
vshape_2_loss.pkl
vshape_2loss.pkl
vshape_2_params.pkl
vshape_2params.pkl
vshape_2.pkl
vshape_30_loss.pkl
vshape_30_params.pkl
vshape_gene_1_loss.pkl
vshape_gene_1_params.pkl


# V-shape

In [8]:
with open('../data/paper/vshape_gene_loss.pkl', 'rb') as handle:
    v_loss = pickle.load(handle)
with open('../data/paper/vshape_gene_params.pkl', 'rb') as handle:
    v_params = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: '../data/paper/vshape_gene_loss.pkl'

# Elongation